In [1]:
import numpy as np
import os
import sys

import h5py
from keras.utils import np_utils, to_categorical
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow import keras
import tensorflow as tf

#model = keras.models.load_model('path/to/location')
# load training and testing ... 

Using TensorFlow backend.


In [2]:
# load training and testing ... 

condition = 'all'

new_train = '..//train//' + condition + '//'
new_test = '..//test//' + condition + '//'
new_val = '..//val//' + condition + '//'

def load_vectors(path):
    files = sorted(os.listdir(path))
    X = np.expand_dims(np.zeros((48, 272)), axis=0)
    y = []
    for npy in files:
        '''
        if 'Calm' in npy:
            continue
        '''
        current = np.load(path+npy)
        X = np.vstack((X, current))
        label = [files.index(npy)]*len(current)       
        y = y + label       
    X = X[1:]
    y = tf.keras.utils.to_categorical(y)    
    print(X.shape)
    print(y.shape)    
    return X, y
    
#X_train, y_train = load_vectors(new_train)
X_test, y_test = load_vectors(new_test)

(9995, 48, 272)
(9995, 6)


In [3]:
from keras.models import Model, load_model, Sequential

model_path = 'D://GitHub//Patient-Caregiver-Relationship//module//five_class_ood//'

model = load_model(model_path + 'model.hdf5')

model_no_softmax = Sequential()
model_last_layer = Sequential()

for index in range(1, len(model.layers)+1):
    if index == len(model.layers):
        model_last_layer.add(model.layers[index-1])
    else:
        model_no_softmax.add(model.layers[index-1])

print('The mood model is intialized.')
del model



The mood model is intialized.


In [4]:
def model_pred_penultimate(fname, model_to_predict):
    if '.wav' not in fname:
        X = [fname]
    else:
        X = [extract_feats_single_wav_to_tensor(fname)]
    y_pred = model_to_predict.predict(np.array(X))

    return y_pred

def model_pred_penultimate_npy(npy, model_to_predict):
    y_pred = model_to_predict.predict(np.asarray([npy]))
    return y_pred

def model_pred_softmax(vec, model_to_predict):
    y_pred = model_to_predict.predict(vec)
    return y_pred

In [5]:
emprical_means_h = np.load(model_path + 'penultimate_mahalanobis_training_samples_emprical_means_h.npy')
emprical_means_a = np.load(model_path + 'penultimate_mahalanobis_training_samples_emprical_means_a.npy')
emprical_means_n = np.load(model_path + 'penultimate_mahalanobis_training_samples_emprical_means_n.npy')
emprical_means_s = np.load(model_path + 'penultimate_mahalanobis_training_samples_emprical_means_s.npy')
emprical_means_o = np.load(model_path + 'penultimate_mahalanobis_training_samples_emprical_means_o.npy')

emprical_std_means_h = np.std(emprical_means_h)
emprical_std_means_a = np.std(emprical_means_a)
emprical_std_means_n = np.std(emprical_means_n)
emprical_std_means_s = np.std(emprical_means_s)
emprical_std_means_o = np.std(emprical_means_o)

emprical_mean_of_means_h = np.mean(emprical_means_h)
emprical_mean_of_means_a = np.mean(emprical_means_a)
emprical_mean_of_means_n = np.mean(emprical_means_n)
emprical_mean_of_means_s = np.mean(emprical_means_s)
emprical_mean_of_means_o = np.mean(emprical_means_o)

means_for_all_classes = [emprical_mean_of_means_h, emprical_mean_of_means_a, emprical_mean_of_means_n, emprical_mean_of_means_s, emprical_mean_of_means_o]
stds_for_all_classes = [emprical_std_means_h, emprical_std_means_a, emprical_std_means_n, emprical_std_means_s, emprical_std_means_o]

mius = np.load(model_path + 'penultimate_emprical_means.npy')

covariance_mat = np.load(model_path + 'penultimate_emprical_covariance.npy')
inverse_covariance = np.linalg.pinv(covariance_mat)
del covariance_mat
print('The emprical class means are loaded.')
print('The covariance matrix is iloaded')

The emprical class means are loaded.
The covariance matrix is iloaded


In [6]:
def min_index(a): 
    a = list(a)
    minpos = a.index(min(a))    
    return minpos

def max_index(a): 
    a = list(a)
    maxpos = a.index(max(a))    
    return maxpos

def calc_mahalanobis(penultimate, C, inverse_covariance, mius):
    mahalanobis_scores = []
    #inverse_covariance = np.linalg.pinv(covariance_mat)
    for c in range(0, len(C)):
        miu = mius[c]
        predicted_minus_class_mean = penultimate - miu
        predicted_minus_class_mean_transpose = np.transpose(predicted_minus_class_mean)
        score = predicted_minus_class_mean_transpose * inverse_covariance * predicted_minus_class_mean_transpose
        score_norm = np.linalg.norm(score)
        mahalanobis_scores.append(score_norm)
    return np.array(mahalanobis_scores)


In [7]:
def test_if_ood_one_sample(fname): # <-- testing ood samples for one npy/wav
    item = model_pred_penultimate(fname, model_no_softmax)
    maha = calc_mahalanobis(item, [0, 1, 2, 3, 4], inverse_covariance, mius)

    plausible = min_index(maha)

    upper = ( means_for_all_classes[plausible] - 0.75 * stds_for_all_classes[plausible] ) < maha[plausible]
    lower = maha[plausible] < ( means_for_all_classes[plausible] + 0.75 * stds_for_all_classes[plausible] )
    
    if upper and lower:
        classification = model_pred_softmax(item, model_last_layer)
        #print('classification: ' + str(np.argmax(classification[0])) )
        return True
    else:
        return False

In [8]:
from tqdm import tqdm

def test_if_ood_samples(X_test, y_test):
    
    new_X = []
    new_Y = []
    
    for X, y in tqdm(zip(X_test, y_test)):
        
        if test_if_ood_one_sample(X):
            new_X.append(X)
            new_Y.append(y)
            
    new_X = np.asarray(new_X)
    new_Y = np.asarray(new_Y)
    return new_X, new_Y
        

In [9]:
new_X, new_Y = test_if_ood_samples(X_test, y_test)

0it [00:00, ?it/s]C:\Users\Ash Gao\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
9995it [2:08:02,  1.30it/s]


In [10]:
np.save('new_X_3rd.npy', new_X)
np.save('new_y_3rd.npy', new_Y)